In [1]:
def cal_bpref(testSet, uid, recommend):
    cited = [] # Doc 'd' cited these paper in test dataset
    rank = []
    for cited_id, rate in testSet[uid].items():    # for name, age in dictionary.iteritems():  (for Python 2.x)
        cited.append(cited_id)
    for i in range(len(recommend)):
        rank.append(int(i+1))
    
    pd.set_option('mode.chained_assignment', None)
    df = pd.DataFrame()
    df['citing'] = recommend
    df['rank'] = rank
    df.insert(2, 'judge', 'Any') 
    judge = []
    judge_r = []

    for rec, ranknum in zip(df['citing'], df['rank']): 
        if rec in cited:
            df['judge'][df['citing'] == rec] = 'r'
        elif (rec not in cited):
            df['judge'][df['citing'] == rec] = 'nr'
#     print(df)
    for c in cited:
        if c not in list(df['citing']):
            judge_r.append(c)
#     print('\nNot in result list:', judge_r, '\n')

    RR = list(df['citing'][df['judge'] == 'r']) # judged relevant in result list
    R = RR + judge_r # judged relevant in result list and not in result list
    N = list(df['citing'][df['judge'] == 'nr'])

    bpref = 0
    for i in RR:
        hit = 0
        for j in range(len(df)):
            if j+1 < int(df['rank'][df['citing'] == i].to_string(index=False)):
                if (df['judge'][df['rank'] == j+1] == 'nr').bool() == True:
                    hit += 1
#         print('|NR(Hit #%s)| = %d' %(i, hit))
        bpref += (1.0 - (hit / len(recommend)))

    if len(R) > 0:
        bpref = bpref / len(R)
#         print('bpref =', bpref)
    
    return bpref

Precision at k is the proportion of recommended items in the top-k set that are relevant

Recall at k is the proportion of relevant items found in the top-k recommendations

Precision@k = (# of recommended items @k that are relevant) / (# of recommended items @k)

Recall@k = (# of recommended items @k that are relevant) / (total # of relevant items)

MRR https://medium.com/swlh/rank-aware-recsys-evaluation-metrics-5191bba16832

In [2]:
def evaluation(dataset_name):
    evaluate_element = ['_precision', '_recall', '_f1', '_rr', '_bpref']
    all_data = []
    data = pd.DataFrame()
    for i in range(1, 6):
        print('*' * 70)
        print('\tThis is eveluated results of %s Fold %i' % (dataset_name, i))
        print('*' * 70 + '\n')
        lst = []
        header = []
        for ele in evaluate_element:
            avg_k = []
            filename = 'data/' + dataset_name + '/evaluation/test_fold'+str(i)+str(ele)+'.csv'
            if ele == '_rr':
                dff_rr = pd.read_csv(filename)
                header.append('MRR')
                lst.append(np.average(dff_rr['rr']))
                print('MRR of this fold:', np.average(dff_rr['rr']))
            elif (ele == '_bpref'):
                dff_bpref = pd.read_csv(filename)
                header.append('Bpref')
                lst.append(np.average(dff_bpref['bpref']))
                print('Avg of bpref of this fold:', np.average(dff_bpref['bpref']))
            else:
                df_eva = pd.read_csv(filename, header = None,index_col=False, sep='\t')
        #         print(df_eva)
                print('Avg of ' + ele[1:] + ' of this fold:')
                for j in range(20):
                    avg_k.append(np.average(df_eva[j]))
                    lst.append(np.average(df_eva[j]))
                    header.append(ele[1:]+'@'+str(j+1))
                print(*avg_k, sep='\t')
    #             print(pd.DataFrame(avg_k).T.to_string(header=False, index = False))
                print()

        all_data.append(lst)
        data['Fold' + str(i)] = lst
    output = 'data/' + dataset_name + '/evaluation.csv'
    data.T.to_csv(output, header=header, index=True)

In [3]:
def combine_file():
    all_dataset_name = ['CiteULike_5_fold', 'CiteSeer_umd_5_fold', 'AlgoCitation_5_fold', 'TopicCitation_5_fold']
    aaa = []
    for dataset in all_dataset_name:
        filename = 'data/' + dataset + '/evaluation.csv'
        dfff = pd.read_csv(filename, sep=',')
        aaa.append(dfff.values.tolist())
    # print(aaa[0][1][3]) # dataset, fold, column

    # write summary of all fold of each dataset into one csv
    with open('all_evalution_CF.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(i for i in dfff.columns)
        for i in aaa:
            writer.writerows(i)

In [16]:
# -*- coding = utf-8 -*-
"""
Main function to build recommendation systems.

Created on 2018-04-16

@author: fuxuemingzhu
"""
import utils
from ItemCF import ItemBasedCF
from LFM import LFM
from UserCF import UserBasedCF
from dataset import DataSet
from most_popular import MostPopular
from random_pred import RandomPredict
from utils import LogTime
from sklearn.model_selection import KFold
import numpy as np
import collections
from collections import namedtuple
import random
import tool
import pandas as pd
import numpy as np
import csv
import os

# def run_model(model_name, dataset_name, test_size=0.3, clean=True):
#     print('*' * 70)
#     print('\tThis is %s model trained on %s with test_size = %.2f' % (model_name, dataset_name, test_size))
#     print('*' * 70 + '\n')
#     model_manager = utils.ModelManager(dataset_name, test_size)
#     '''if you want to change test_size or retrain model, please set clean_workspace True'''
#     model_manager.clean_workspace(clean)
    
#     if model_name == 'UserCF':
#         model = UserBasedCF()
#     elif model_name == 'ItemCF':
#         model = ItemBasedCF()
#     elif model_name == 'Random':
#         model = RandomPredict()
#     elif model_name == 'MostPopular':
#         model = MostPopular()
#     elif model_name == 'UserCF-IIF':
#         model = UserBasedCF(use_iif_similarity=True, save_model=False)
#     elif model_name == 'ItemCF-IUF':
#         model = ItemBasedCF(use_iuf_similarity=True)
#     elif model_name == 'LFM':
#         # K, epochs, alpha, lamb, n_rec_movie
#         model = LFM(10, 20, 0.1, 0.01, 10)
#     else:
#         raise ValueError('No model named ' + model_name)
        
#     summary = {}
#     precision = 0
#     recall = 0
#     coverage = 0
#     popularity = 0
#     for i in range(1, 6):
#         filename = []
#         paper_id = []
#         bpref = []
#         rr = [] 
#         trainSet = tool.loadData("data/" + dataset_name + "/train_fold" + str(i)+'.txt')
#         testSet = tool.loadData("data/" + dataset_name + "/test_fold" + str(i)+'.txt')
# #         print(testSet)
#         model.fit(trainSet)
        
#         for key in testSet: # paper id in test dataset of each fold
#             paper_id.append(key)
#             recommend = model.recommend(str(key))
#             bpref.append(cal_bpref(testSet, key, recommend))
            
#             # precision@k, recall@k, f1@k, mrr
#             hit = 0
#             precision_k = []
#             recall_k = []
#             f1_score_k = []
            
#             rec_count = 0
#             test_count = 0
#             test_paper = testSet.get(key, {}) #real answer recommend list
#             test_count = len(test_paper)
#             rrank = 0
            
#             # print(test_paper)
#             for pid in recommend:
#                 if pid in test_paper:
#                     hit += 1
#                     if hit == 1:
#                         rrank = rec_count + 1
#                 rec_count += 1
                
#                 if len(recommend) == 0:
#                     precision_k.append(0)
#                     recall_k.append(0)
#                     f1_score_k.append(0)
#                 else:
#                     pre_temp = hit / (1.0 * rec_count)
#                     rec_temp = hit / (1.0 * test_count)
#                     if (pre_temp + rec_temp) == 0:
#                         f1_temp = 0
#                     else:
#                         f1_temp = 2 * ((pre_temp * rec_temp) / (pre_temp + rec_temp))
                        
#                     precision_k.append(pre_temp)
#                     recall_k.append(rec_temp)
#                     f1_score_k.append(f1_temp)
                    
#             if rrank != 0:
#                 rr.append(1 / rrank)
#             else:
#                 rr.append(0)
                
#             if len(precision_k) != 10:
#                 for k in range(10):
#                     precision_k.append(0.0)
#                     if len(precision_k) == 10:
#                         break
#             if len(recall_k) != 10:
#                 for k in range(10):
#                     recall_k.append(rec_temp)
#                     if len(recall_k) == 10:
#                         break
#             if len(f1_score_k) != 10:
#                 for k in range(10):
#                     f1_score_k.append(0.0)
#                     if len(f1_score_k) == 10:
#                         break
                    
#             with open('data/' + dataset_name + '/evaluation/test_fold'+str(i)+'_precision.csv','a') as f1:
#                 writer=csv.writer(f1, delimiter='\t')#lineterminator='\n',
#                 row = precision_k
#                 writer.writerow(row)
            
#             with open('data/' + dataset_name + '/evaluation/test_fold'+str(i)+'_recall.csv','a') as f1:
#                 writer=csv.writer(f1, delimiter='\t')#lineterminator='\n',
#                 row = recall_k
#                 writer.writerow(row)
#             with open('data/' + dataset_name + '/evaluation/test_fold'+str(i)+'_f1.csv','a') as f1:
#                 writer=csv.writer(f1, delimiter='\t')#lineterminator='\n',
#                 row = f1_score_k
#                 writer.writerow(row)

#         print('Mean of bpref for ' + dataset_name + '_test_fold' + str(i) + ':', np.mean(bpref))
#         # Save bpref to csv
#         df_bpref = pd.DataFrame()
#         df_bpref['key'] = paper_id
#         df_bpref['bpref'] = bpref
#         df_bpref.to_csv("data/" + dataset_name + "/evaluation/test_fold" + str(i) + "_bpref.csv",index=False)
#         # Save bpref to csv
#         df_rr = pd.DataFrame()
#         df_rr['key'] = paper_id
#         df_rr['rr'] = rr
#         df_rr.to_csv('data/' + dataset_name + '/evaluation/test_fold'+str(i)+'_rr.csv',index=False)


# #         recommend_test(model, [356,218,527, 273])
# #         model.test(testSet)
#         results = model.test(testSet)
#         precision += results[0]
#         recall += results[1]
#         coverage += results[2]
#         popularity += results[3]
#     print('FINAL precision=%.4f\trecall=%.4f\tcoverage=%.4f\tpopularity=%.4f\n' %
#               (precision/5, recall/5, coverage/5, popularity/5))

def run_model(model_name, dataset_name, test_size=0.3, clean=True):
    print('*' * 70)
    print('\tThis is %s model trained on %s with test_size = %.2f' % (model_name, dataset_name, test_size))
    print('*' * 70 + '\n')
    model_manager = utils.ModelManager(dataset_name, test_size)
    '''if you want to change test_size or retrain model, please set clean_workspace True'''
    model_manager.clean_workspace(clean)
    
    if model_name == 'UserCF':
        model = UserBasedCF()
    elif model_name == 'ItemCF':
        model = ItemBasedCF()
    elif model_name == 'Random':
        model = RandomPredict()
    elif model_name == 'MostPopular':
        model = MostPopular()
    elif model_name == 'UserCF-IIF':
        model = UserBasedCF(use_iif_similarity=True, save_model=False)
    elif model_name == 'ItemCF-IUF':
        model = ItemBasedCF(use_iuf_similarity=True)
    elif model_name == 'LFM':
        # K, epochs, alpha, lamb, n_rec_movie
        model = LFM(10, 20, 0.1, 0.01, 10)
    else:
        raise ValueError('No model named ' + model_name)
        
    summary = {}
    precision = 0
    recall = 0
    coverage = 0
    popularity = 0
    for i in range(1, 6):
        path_dir = 'data/' + dataset_name + '/evaluation/test_fold'+str(i)
        eva_metrics = ['_precision.csv', '_recall.csv', '_f1.csv', '_bpref.csv', '_rr.csv']
        for d in eva_metrics:
            print(path_dir + str(d))
            if os.path.exists(path_dir + str(d)):
                os.remove(path_dir + str(d))
            else:
                print('File does not exists')

        filename = []
        paper_id = []
        bpref = []
        rr = [] 
        trainSet = tool.loadData("data/" + dataset_name + "/train_fold" + str(i)+'.txt')
        testSet = tool.loadData("data/" + dataset_name + "/test_fold" + str(i)+'.txt')
#         print(testSet)
        model.fit(trainSet)
        
        for key in testSet: # paper id in test dataset of each fold
            paper_id.append(key)
            recommend = model.recommend(str(key))
            bpref.append(cal_bpref(testSet, key, recommend))
            
            # precision@k, recall@k, f1@k, mrr
            hit = 0
            precision_k = []
            recall_k = []
            f1_score_k = []
            
            rec_count = 0
            test_count = 0
            test_paper = testSet.get(key, {}) #real answer recommend list
            test_count = len(test_paper)
            rrank = 0
            print(len(recommend))
            for x in range(1,21):
                if len(recommend) < 20:
                    recommend.append(str(1000000+x))
                if len(recommend) == 20:
                    break
            print('After add len of recommend:', len(recommend))
            # for calculate RR
            for pid in recommend:
                if pid in test_paper:
                    hit += 1
                    if hit == 1:
                        rrank = rec_count + 1
                rec_count += 1
                
            if rrank != 0:
                rr.append(1 / rrank)
            else:
                rr.append(0)
            # end of calculating RR
                
            for k in range(1, 21):
                pr = len(set(test_paper) & set(recommend[:k])) / k
                rec = len(set(test_paper) & set(recommend[:k])) / len(test_paper)
                F1 = (2 * pr * rec) / (pr + rec) if pr + rec > 0 else 0
                precision_k.append(pr)
                recall_k.append(rec)
                f1_score_k.append(F1)
      
            with open('data/' + dataset_name + '/evaluation/test_fold'+str(i)+'_precision.csv','a') as f1:
                writer=csv.writer(f1, delimiter='\t')#lineterminator='\n',
                row = precision_k
                writer.writerow(row)
            
            with open('data/' + dataset_name + '/evaluation/test_fold'+str(i)+'_recall.csv','a') as f1:
                writer=csv.writer(f1, delimiter='\t')#lineterminator='\n',
                row = recall_k
                writer.writerow(row)
            with open('data/' + dataset_name + '/evaluation/test_fold'+str(i)+'_f1.csv','a') as f1:
                writer=csv.writer(f1, delimiter='\t')#lineterminator='\n',
                row = f1_score_k
                writer.writerow(row)

        print('Mean of bpref for ' + dataset_name + '_test_fold' + str(i) + ':', np.mean(bpref))
        # Save bpref to csv
        df_bpref = pd.DataFrame()
        df_bpref['key'] = paper_id
        df_bpref['bpref'] = bpref
        df_bpref.to_csv("data/" + dataset_name + "/evaluation/test_fold" + str(i) + "_bpref.csv",index=False)
        # Save bpref to csv
        df_rr = pd.DataFrame()
        df_rr['key'] = paper_id
        df_rr['rr'] = rr
        df_rr.to_csv('data/' + dataset_name + '/evaluation/test_fold'+str(i)+'_rr.csv',index=False)


#         recommend_test(model, [356,218,527, 273])
#         model.test(testSet)
        results = model.test(testSet)
        precision += results[0]
        recall += results[1]
        coverage += results[2]
        popularity += results[3]
    print('FINAL precision=%.4f\trecall=%.4f\tcoverage=%.4f\tpopularity=%.4f\n' %
              (precision/5, recall/5, coverage/5, popularity/5))

def recommend_test(model, user_list):
    for user in user_list:
        recommend = model.recommend(str(user))
        print("recommend for userid = %s:" % user)
        print(recommend)
        print()

main_time = LogTime(words="Main Function")
# dataset_name = 'ml-100k'
# dataset_name = 'ml-1m'
# dataset_name = 'cite_u_like'
# dataset_name = 'citeseer_umd'

dataset_name = 'CiteULike_5_fold'
# dataset_name = 'CiteSeer_umd_5_fold'
# dataset_name = 'AlgoCitation_5_fold'
# dataset_name = 'TopicCitation_5_fold'
# model_type = 'UserCF'
model_type = 'UserCF-IIF'
# model_type = 'ItemCF'
# model_type = 'Random'
# model_type = 'MostPopular'
# model_type = 'ItemCF-IUF'
# model_type = 'LFM'

test_size = 0.1
# run_model(model_type, dataset_name, test_size, False)
# evaluation(dataset_name)
dataset_name = ['CiteULike_5_fold', 'CiteSeer_umd_5_fold', 'AlgoCitation_5_fold', 'TopicCitation_5_fold']
for data in dataset_name:
    run_model(model_type, data, test_size, False)
    evaluation(data)
main_time.finish()

**********************************************************************
	This is UserCF-IIF model trained on CiteULike_5_fold with test_size = 0.10
**********************************************************************

UserBasedCF start...

data/CiteULike_5_fold/evaluation/test_fold1_precision.csv
data/CiteULike_5_fold/evaluation/test_fold1_recall.csv
data/CiteULike_5_fold/evaluation/test_fold1_f1.csv
data/CiteULike_5_fold/evaluation/test_fold1_bpref.csv
data/CiteULike_5_fold/evaluation/test_fold1_rr.csv
No model saved before.
Train a new model...
building movie-users inverse table...
building movie-users inverse table success.
total movie number = 3777
generate user co-rated movies similarity matrix...
 steps(0), 0.00 seconds have spent..
 steps(1000), 0.03 seconds have spent..
 steps(2000), 0.03 seconds have spent..
 steps(3000), 0.03 seconds have spent..
generate user co-rated movies similarity matrix success.
total  step number is 3777
total 0.03 seconds have spent

calculate user-

In [17]:
combine_file()